In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

import os
path = "/content/drive/MyDrive/semester 2/AM216/HW4" # Your path here
os.chdir(path)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# This is how the triagle lattice data is generated. You may find it helpful to generate some 
# of your own data
from __future__ import division
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt

class Ising_tri():
    ''' Simulating the Ising model '''  
    def __init__(self, size, temp):
        self.temp = temp
        self.N = int(size)
    ## monte carlo moves
    def mcmove(self, config, N, beta):
        ''' This is to execute the monte carlo moves using 
        Metropolis algorithm such that detailed
        balance condition is satisified'''
        for i in range(N):
            for j in range(N):            
                    a = np.random.randint(0, N) # select a row
                    b = np.random.randint(0, N) # select a column
                    s =  config[a, b] # current state at (a, b)
                    if a%2:
                        nb = config[(a+1)%N,b] +config[(a+1)%N,(b+1)%N] + config[a,(b+1)%N] + \
                        config[(a-1)%N,b] + config[(a-1)%N,(b+1)%N] + config[a,(b-1)%N]
                    else:
                        nb = config[(a+1)%N,b] +config[(a+1)%N,(b-1)%N] + config[a,(b+1)%N] + \
                        config[(a-1)%N,b] + config[(a-1)%N,(b-1)%N] + config[a,(b-1)%N]
                    
                    
                    cost = 2*s*nb
                    if cost < 0:	
                        s *= -1
                    elif rand() < np.exp(-cost*beta):
                        s *= -1
                    config[a, b] = s
        return config
    
    def simulate(self):   
        ''' This module simulates the Ising model'''
        config = 2*np.random.randint(2, size=(self.N,self.N))-1   
        msrmnt = 81
        for i in range(msrmnt):
            self.mcmove(config, self.N, 1.0/self.temp)
        return config

You can import 4-temp data for square and triangular lattices as follows

In [ ]:
N = 250
nx, ny = 32, 32

Xsq = np.ndarray((4*N,nx,ny,1))
ysq = np.ndarray(4*N)

for i in np.arange(N):
    Xsq[i + 0*N] = np.loadtxt("./square_T1/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 0*N] = 0
    Xsq[i + 1*N] = np.loadtxt("./square_T2/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 1*N] = 1
    Xsq[i + 2*N] = np.loadtxt("./square_T3/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 2*N] = 2
    Xsq[i + 3*N] = np.loadtxt("./square_T4/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 3*N] = 3

Xsq_train, Xsq_test, ysq_train, ysq_test = train_test_split(Xsq, ysq, test_size=0.2, random_state=0)

In [ ]:
N = 250
nx, ny = 32, 32

Xtri = np.ndarray((4*N,nx,ny,1))
ytri = np.ndarray(4*N)

for i in np.arange(N):
    Xtri[i + 0*N] = np.loadtxt("./triangle_T1/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 0*N] = 0
    Xtri[i + 1*N] = np.loadtxt("./triangle_T2/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 1*N] = 1
    Xtri[i + 2*N] = np.loadtxt("./triangle_T3/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 2*N] = 2
    Xtri[i + 3*N] = np.loadtxt("./triangle_T4/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 3*N] = 3

Xtri_train, Xtri_test, ytri_train, ytri_test = train_test_split(Xtri, ytri, test_size=0.2, random_state=0)

Make sure you know the shape of data.

In [ ]:
print("Shape of training data:")
print(Xsq_train.shape, Xtri_train.shape)
print(ysq_train.shape, ytri_train.shape)
print("Shape of test data:")
print(Xsq_test.shape, Xtri_test.shape)
print(ysq_test.shape, ytri_test.shape)

Shape of training data:
(800, 32, 32, 1) (800, 32, 32, 1)
(800,) (800,)
Shape of test data:
(200, 32, 32, 1) (200, 32, 32, 1)
(200,) (200,)


### (a) Train a fully connected neural network to do the classification on both datasets. Then, train  a  convolutional  neural  network  to  do  the  classification,  on  both datasets.   Make  a  table  of  your  performance  numbers  for  both  models  and  upload  these  numbers.   This,  together  with  your code,  should be uploaded to the course website when you turn in yourhomework.

The temperatures for square lattice are $T = 1.5, 2.1, 2.4, 3.5$. $T = 2.5, 3.2, 3.8, 5$ for triangle lattice.


Solution to (a):

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# Fully connected NN 
sq_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(32, 32,1)),
   BatchNormalization(),
   tf.keras.layers.Dense(128, activation=tf.nn.relu),  
   tf.keras.layers.Dropout(0.2),   
    BatchNormalization(), 
    tf.keras.layers.Dense(64, activation=tf.nn.relu),  
  tf.keras.layers.Dense(4, activation=tf.nn.softmax) # output layer to predict for 4 classes
])

sq_model.compile(
    optimizer=tf.keras.optimizers.Adagrad(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

Xsq_fit = sq_model.fit(Xsq_train, ysq_train, epochs = 10)

Epoch 1/10
25/25 [==============================] - 3s 4ms/step - loss: 1.4191 - sparse_categorical_accuracy: 0.2525
Epoch 2/10
25/25 [==============================] - 0s 4ms/step - loss: 1.4044 - sparse_categorical_accuracy: 0.2600
Epoch 3/10
25/25 [==============================] - 0s 4ms/step - loss: 1.3985 - sparse_categorical_accuracy: 0.2788
Epoch 4/10
25/25 [==============================] - 0s 4ms/step - loss: 1.3985 - sparse_categorical_accuracy: 0.2587
Epoch 5/10
25/25 [==============================] - 0s 4ms/step - loss: 1.4029 - sparse_categorical_accuracy: 0.2763
Epoch 6/10
25/25 [==============================] - 0s 4ms/step - loss: 1.3920 - sparse_categorical_accuracy: 0.2763
Epoch 7/10
25/25 [==============================] - 0s 4ms/step - loss: 1.3851 - sparse_categorical_accuracy: 0.3125
Epoch 8/10
25/25 [==============================] - 0s 4ms/step - loss: 1.3925 - sparse_categorical_accuracy: 0.2750
Epoch 9/10
25/25 [==============================] - 0s 4ms/step 

In [ ]:
# Fully connected NN - triangle
tri_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(32, 32,1)),
  BatchNormalization(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),  # first layer with activation function
  tf.keras.layers.Dropout(0.2),   
    BatchNormalization(), 
    tf.keras.layers.Dense(64, activation=tf.nn.relu),  # first layer with activation function
  tf.keras.layers.Dense(4, activation=tf.nn.softmax) # output layer to predict for 4 classes
])

tri_model.compile(
    optimizer=tf.keras.optimizers.Adagrad(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
Xtri_fit = tri_model.fit(Xtri_train, ytri_train, epochs = 10)

Epoch 1/10
25/25 [==============================] - 1s 4ms/step - loss: 1.4149 - sparse_categorical_accuracy: 0.2500
Epoch 2/10
25/25 [==============================] - 0s 4ms/step - loss: 1.4089 - sparse_categorical_accuracy: 0.2537
Epoch 3/10
25/25 [==============================] - 0s 4ms/step - loss: 1.4039 - sparse_categorical_accuracy: 0.2750
Epoch 4/10
25/25 [==============================] - 0s 4ms/step - loss: 1.3943 - sparse_categorical_accuracy: 0.2775
Epoch 5/10
25/25 [==============================] - 0s 4ms/step - loss: 1.3856 - sparse_categorical_accuracy: 0.3000
Epoch 6/10
25/25 [==============================] - 0s 4ms/step - loss: 1.3868 - sparse_categorical_accuracy: 0.2950
Epoch 7/10
25/25 [==============================] - 0s 4ms/step - loss: 1.3843 - sparse_categorical_accuracy: 0.3000
Epoch 8/10
25/25 [==============================] - 0s 4ms/step - loss: 1.3755 - sparse_categorical_accuracy: 0.3162
Epoch 9/10
25/25 [==============================] - 0s 4ms/step 

In [ ]:
sq_loss, sq_acc = sq_model.evaluate(Xsq_test, ysq_test)
tri_loss, tri_acc = tri_model.evaluate(Xtri_test, ytri_test)
print()
print(f"Square Model loss: {sq_loss} & accuracy {sq_acc}" )
print(f"Triangle Model loss: {tri_loss} & accuracy {tri_acc}")

7/7 [==============================] - 0s 4ms/step - loss: 1.3732 - sparse_categorical_accuracy: 0.3000

Square Model loss: 1.4034533500671387 & accuracy 0.25999999046325684
Triangle Model loss: 1.3732342720031738 & accuracy 0.30000001192092896


### (b) Train a convolutional neural network to do the classification, on both datasets. Make a table of your performance numbers for (a) and (b). 
Try to optimize the performance of your models and compare the result.

solution to (b):

In [ ]:
cnn_square = tf.keras.models.Sequential([ # CNN layers
                                        tf.keras.layers.Conv2D(32, (3, 3), activation=tf.nn.relu, input_shape=(32, 32, 1)),
                                        tf.keras.layers.MaxPooling2D((2, 2)),
                                        tf.keras.layers.Dropout(0.2),
                                        tf.keras.layers.Conv2D(64, (3, 3), activation=tf.nn.relu),
                                        tf.keras.layers.MaxPooling2D((2, 2)), 
                                        tf.keras.layers.Dropout(0.2),
                                        tf.keras.layers.Conv2D(64, (3, 3), activation=tf.nn.relu),
                                        # output layers to use for classification 
                                        tf.keras.layers.Flatten(),
                                        tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                        tf.keras.layers.Dense(4)
                                        ])



In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
cnn_square.compile(optimizer='adam',     # This object specifies the training procedure
                  loss=loss_fn,         # The function to minimize during optimization
                  metrics=['accuracy'])

Xsq_cnn = cnn_square.fit(Xsq_train, ysq_train, epochs=10)

Epoch 1/10
25/25 [==============================] - 8s 9ms/step - loss: 1.0255 - accuracy: 0.5663
Epoch 2/10
25/25 [==============================] - 0s 8ms/step - loss: 0.3807 - accuracy: 0.8462
Epoch 3/10
25/25 [==============================] - 0s 8ms/step - loss: 0.2578 - accuracy: 0.8988
Epoch 4/10
25/25 [==============================] - 0s 8ms/step - loss: 0.2165 - accuracy: 0.9250
Epoch 5/10
25/25 [==============================] - 0s 8ms/step - loss: 0.2113 - accuracy: 0.9200
Epoch 6/10
25/25 [==============================] - 0s 8ms/step - loss: 0.1706 - accuracy: 0.9325
Epoch 7/10
25/25 [==============================] - 0s 8ms/step - loss: 0.1393 - accuracy: 0.9525
Epoch 8/10
25/25 [==============================] - 0s 8ms/step - loss: 0.1306 - accuracy: 0.9388
Epoch 9/10
25/25 [==============================] - 0s 8ms/step - loss: 0.1130 - accuracy: 0.9600
Epoch 10/10
25/25 [==============================] - 0s 8ms/step - loss: 0.1418 - accuracy: 0.9538


In [ ]:
cnnsq_loss, cnnsq_acc = cnn_square.evaluate(Xsq_test, ysq_test)

7/7 [==============================] - 0s 12ms/step - loss: 0.3308 - accuracy: 0.8750


In [ ]:
cnn_triangle = tf.keras.models.Sequential([ # CNN layers
                                        tf.keras.layers.Conv2D(32, (3, 3), activation=tf.nn.relu, input_shape=(32, 32, 1)),
                                        tf.keras.layers.MaxPooling2D((2, 2)),
                                        tf.keras.layers.Dropout(0.2),
                                        tf.keras.layers.Conv2D(64, (3, 3), activation=tf.nn.relu),
                                        tf.keras.layers.MaxPooling2D((2, 2)), 
                                        tf.keras.layers.Dropout(0.2),
                                        tf.keras.layers.Conv2D(64, (3, 3), activation=tf.nn.relu),
                                        # output layers to use for classification 
                                        tf.keras.layers.Flatten(),
                                        tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                        tf.keras.layers.Dense(4)
                                        ])


loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
cnn_triangle.compile(optimizer='adam',     # This object specifies the training procedure
                  loss=loss_fn,         # The function to minimize during optimization
                  metrics=['accuracy'])


cnn_triangle.fit(Xtri_train, ytri_train, epochs = 12)

Epoch 1/12
25/25 [==============================] - 1s 8ms/step - loss: 1.1576 - accuracy: 0.4563
Epoch 2/12
25/25 [==============================] - 0s 8ms/step - loss: 0.5221 - accuracy: 0.7550
Epoch 3/12
25/25 [==============================] - 0s 8ms/step - loss: 0.3309 - accuracy: 0.8662
Epoch 4/12
25/25 [==============================] - 0s 8ms/step - loss: 0.2963 - accuracy: 0.8775
Epoch 5/12
25/25 [==============================] - 0s 8ms/step - loss: 0.3881 - accuracy: 0.8163
Epoch 6/12
25/25 [==============================] - 0s 8ms/step - loss: 0.2201 - accuracy: 0.9212
Epoch 7/12
25/25 [==============================] - 0s 8ms/step - loss: 0.1903 - accuracy: 0.9200
Epoch 8/12
25/25 [==============================] - 0s 8ms/step - loss: 0.1393 - accuracy: 0.9613
Epoch 9/12
25/25 [==============================] - 0s 8ms/step - loss: 0.1567 - accuracy: 0.9375
Epoch 10/12
25/25 [==============================] - 0s 8ms/step - loss: 0.1394 - accuracy: 0.9575
Epoch 11/12
25/25 [

In [ ]:
cnntri_loss, cnntri_acc = cnn_triangle.evaluate(Xtri_test, ytri_test)

7/7 [==============================] - 0s 5ms/step - loss: 0.1617 - accuracy: 0.9300


In [ ]:
import pandas as pd
# square / triangle
scores = {"Connected Neural Network":[tri_acc,sq_acc] , "Convolutional Neural Network":[cnntri_acc,cnnsq_acc]}
df = pd.DataFrame.from_dict(scores, orient = 'index',columns = {"triangle", "square"})
print("Table of the Accuracy Scores")
df

Table of the Accuracy Scores


,square,triangle
Connected Neural Network,0.30,0.260
Convolutional Neural Network,0.93,0.875


### (c) We have provided a test set of 10 spins configurations for each of the two problems. Each of the spin configurations is not necessarily at the temperatures of the training sets. Calculate your best estimate of the temperatures of these spin configuration. Upload your results to Kaggle.
[Hint: A direct fingerprint of temperature is the distribution of spin up
and down, because you can image that the spins fluctuate more violently
at higher temperature. Although the mothod you use in homework 2 can also work, you may be interested in trying to take distribution into account when you
build the model to estimate temperature and see if you can make use of this extra information. This may help you win the
kaggle. It is totally fine if you find that the information of distribution is not helpful. Note also that a CNN kind-of does this. One possibility is that you may want a CNN that captures enough distribution information.]

Solution to (c)

In [ ]:
# download from websites
N = 100
nx, ny = 32, 32

Xsq = np.ndarray((N,nx,ny,1))
ysq = np.ndarray(N)

val = 0
for i in np.arange(10):
    for j in np.arange(10):
        Xsq[val] = np.loadtxt(f"square_10T/T0{i}#0{j}".format(i), delimiter=",").reshape(nx,ny,1)
        ysq[val] = 0
        val += 1
    
Xsq.shape

(100, 32, 32, 1)

In [ ]:
Xsq_pred = cnn_square.predict(Xsq)
Xsq_pred.shape

(100, 4)

In [ ]:
predictions = np.zeros((100,1))
for i in range(100):
    predictions[i] = np.argmax(Xsq_pred[i])
    
total_guess = np.zeros((10,1))
start = 0
end = 10
for i in range(10):
    total_guess[i] = round(np.sum(predictions[start:end])/10,0)
    start = end
    end = start + 10
    
total_guess

array([[3.],
       [3.],
       [1.],
       [0.],
       [0.],
       [3.],
       [3.],
       [1.],
       [3.],
       [0.]])

In [ ]:
conversion = [1.5,2.1,2.4,3.5]
temps = []

for i in total_guess:
    temps.append(conversion[int(i)])
print("Guesses for Square")   
print(temps)

Guesses for Square
[3.5, 3.5, 2.1, 1.5, 1.5, 3.5, 3.5, 2.1, 3.5, 1.5]


In [ ]:
N = 100
nx, ny = 32, 32

Xtri = np.ndarray((N,nx,ny,1))
ytri = np.ndarray(N)

val = 0
for i in np.arange(10):
    for j in np.arange(10):
        Xtri[val] = np.loadtxt(f"triangle_10T/T0{i}#0{j}".format(i), delimiter=",").reshape(nx,ny,1)
        ytri[val] = 0
        val += 1
    
Xtri.shape

(100, 32, 32, 1)

In [ ]:
Xtri_pred = cnn_triangle.predict(Xtri)
Xtri_pred.shape

(100, 4)

In [ ]:
pred = np.zeros((100,1))
for i in range(100):
    pred[i] = np.argmax(Xsq_pred[i])
    
guess_tri = np.zeros((10,1))
start = 0
end = 10
for i in range(10):
    guess_tri[i] = round(np.sum(pred[start:end])/10,0)
    start = end
    end = start + 10
    
guess_tri

array([[3.],
       [3.],
       [1.],
       [0.],
       [0.],
       [3.],
       [3.],
       [1.],
       [3.],
       [0.]])

In [ ]:
tri_conv = [2.5,3.2,3.8,5 ]
temps = []

for i in guess_tri:
    temps.append(tri_conv[int(i)])
print("Guesses for Triangle Lattice")   
print(temps)

Guesses for Triangle Lattice
[5, 5, 3.2, 2.5, 2.5, 5, 5, 3.2, 5, 2.5]


### (d) *Transfer Learning*.  
As we emphasize in class, one can freeze the training of the bottom layers of a network and retrain the top part of the network to adopt to a new situation. Use your CNN that you trained on the squarelattice data to do transfer learning on the triangular lattice data.  How does the performance compare to that of the direct methods?  Add the performance numbers for transfer learning in your table from Part (a). Note that the training time and number of training examples needed for transfer learning is far less than that for the direct  optimization. For  example,  is  50  triangle  example  sufficient  for the re-training process?  Use your transfer learning result to predict the transition temperature of triangle lattice Ising model, as demonstrated in this [Nature Physics](https://www-nature-com.ezp-prod1.hul.harvard.edu/articles/nphys4035.pdf) publication.

As a guideline, you may like to just change the last `Dense` layer with `softmax` activation when you do the transfer learning. Other choices are also OK.

Solution to (d):

In [ ]:
cnn_square.trainable = False
cnn_square.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout_2 (Dropout)         (None, 15, 15, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 6, 6, 64)         

In [ ]:
# put it all together into a model.
prediction_layer = tf.keras.layers.Dense(4)

inputs = tf.keras.Input(shape=(32, 32, 1))
x = cnn_square(inputs, training=False)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(Xtri_train, ytri_train,epochs=30)

Epoch 1/30
25/25 [==============================] - 1s 4ms/step - loss: 5.8038 - accuracy: 0.3812
Epoch 2/30
25/25 [==============================] - 0s 4ms/step - loss: 5.3303 - accuracy: 0.4050
Epoch 3/30
25/25 [==============================] - 0s 4ms/step - loss: 4.4234 - accuracy: 0.4300
Epoch 4/30
25/25 [==============================] - 0s 4ms/step - loss: 4.0383 - accuracy: 0.4387
Epoch 5/30
25/25 [==============================] - 0s 4ms/step - loss: 3.5649 - accuracy: 0.4588
Epoch 6/30
25/25 [==============================] - 0s 4ms/step - loss: 2.9799 - accuracy: 0.4500
Epoch 7/30
25/25 [==============================] - 0s 4ms/step - loss: 2.5310 - accuracy: 0.4850
Epoch 8/30
25/25 [==============================] - 0s 4ms/step - loss: 2.6754 - accuracy: 0.4700
Epoch 9/30
25/25 [==============================] - 0s 4ms/step - loss: 2.3204 - accuracy: 0.5063
Epoch 10/30
25/25 [==============================] - 0s 4ms/step - loss: 2.0547 - accuracy: 0.5213
Epoch 11/30
25/25 [

In [ ]:
model.evaluate(Xtri_test, ytri_test)

7/7 [==============================] - 0s 4ms/step - loss: 0.3161 - accuracy: 0.8800


[0.31609952449798584, 0.8799999952316284]

In [ ]:
# predict transition temperatire
# temperature for T = 2:5; 3:2; 3:8; 5
og_temp = [2.5,2.3,3.8,5]
shape = Xtri_test[0].shape[0]
ising = Ising_tri(shape,5).simulate()
ising.shape

(32, 32)

In [ ]:
# Temperatures I want to test to see how the model predicts
test_temps = np.linspace(2.5,5,10)
test_temps

array([2.5       , 2.77777778, 3.05555556, 3.33333333, 3.61111111,
       3.88888889, 4.16666667, 4.44444444, 4.72222222, 5.        ])

In [ ]:
new_test = np.zeros((100,32,32))

for i, temp in enumerate(test_temps):
  for j in range(10):
    ising = Ising_tri(shape,temp).simulate()
    new_test[i+j] = ising

In [ ]:
new_labels = np.zeros((100,))
j = 0
start = 0
end = 10
for i in range(10):
  new_labels[start:end] = test_temps[j]
  start = end
  end = start + 10
  j +=1

In [ ]:
t0_data = new_test[:10]
t1_data = new_test[10:20]
t2_data = new_test[20:30]
t3_data = new_test[30:40]
t4_data = new_test[40:50]
t5_data = new_test[50:60]
t6_data = new_test[60:70]
t7_data = new_test[70:80]
t8_data = new_test[80:90]
t9_data = new_test[90:100]

In [ ]:
from operator import truediv
cnn_square.trainable = True
cnn_square.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout_2 (Dropout)         (None, 15, 15, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 6, 6, 64)         

In [ ]:
prediction_layer = tf.keras.layers.Dense(2)

inputs = tf.keras.Input(shape=(32, 32, 1))
x = cnn_square(inputs, training=True)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 1)]       0         
                                                                 
 sequential_2 (Sequential)   (None, 4)                 121604    
                                                                 
 dropout_7 (Dropout)         (None, 4)                 0         
                                                                 
 dense_11 (Dense)            (None, 2)                 10        
                                                                 
Total params: 121,614
Trainable params: 121,614
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# Try to classify two at a time
# t0 and t1 [t0_data, t1_data] new_labels[:20]

t01 = np.vstack((t0_data, t1_data))
model.fit(t0_data, new_labels[:10], epochs = 10)

array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]], dtype=float32)